In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [953 kB]
Get:13 http://archive.

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-17 15:01:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-04-17 15:01:19 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

### Create DataFrames to match tables

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R215C9BDXTDQOW|          4|            0|          0|   N|                Y|
|R1DTPUV1J57YHA|          5|            0|          0|   N|                Y|
| RFAZK5EWKJWOU|          5|            0|          0|   N|                Y|
|R2XT8X0O0WS1AL|          5|            0|          0|   N|                Y|
|R14GRNANKO2Y2J|          5|            0|          0|   N|                Y|
|R2BLF9VYL24LCQ|          5|            1|          1|   N|                Y|
|R1GI9UW5KJ671O|          5|            0|          0|   N|                Y|
|R2H5CEJN863M86|          5|            0|          1|   N|                Y|
| R5PPDHFOZ3SMU|          5|            0|          0|   N|                Y|
| RE1L9IENKJJ7Y|          1|            0|          0|   N|     

DELIVERABLE 2

In [6]:
filtered_vine_df = vine_df.filter(vine_df.total_votes>19)
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [7]:
helpful_votes_df = filtered_vine_df.withColumn('Helpful_Total_Votes',filtered_vine_df['helpful_votes']/filtered_vine_df['total_votes']).filter(('Helpful_Total_Votes>=.5') )
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|                1.0|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y| 0.9555555555555556|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y| 0.8873239436619719|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y| 0.9655172413793104|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y| 0.9142857142857143|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y| 0.9217391304347826|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y| 0.9705882352941176|
| RQCC7XNYI014B|    

In [8]:
vine_votes_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R29V4UDSS053D8|          3|           33|         38|   Y|                N|  0.868421052631579|
|R1I2D068WC37PA|          3|           32|         37|   Y|                N| 0.8648648648648649|
|R13W2U74F67QED|          5|           62|         76|   Y|                N| 0.8157894736842105|
|R2QI37XFOBKUGD|          4|           24|         33|   Y|                N| 0.7272727272727273|
|R19F60BB2DNCKN|          5|          170|        174|   Y|                N| 0.9770114942528736|
|R2PYNEWYYQDWHI|          5|           24|         27|   Y|                N| 0.8888888888888888|
|R1UEWHCPJL2XJB|          5|           70|         72|   Y|                N| 0.9722222222222222|
| RXTMMEBRYZB53|    

In [9]:
vine_votes_df.count()

266

In [10]:
vine_5_star = vine_votes_df.filter(vine_votes_df.star_rating == 5)
vine_5_star.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R13W2U74F67QED|          5|           62|         76|   Y|                N| 0.8157894736842105|
|R19F60BB2DNCKN|          5|          170|        174|   Y|                N| 0.9770114942528736|
|R2PYNEWYYQDWHI|          5|           24|         27|   Y|                N| 0.8888888888888888|
|R1UEWHCPJL2XJB|          5|           70|         72|   Y|                N| 0.9722222222222222|
| RXTMMEBRYZB53|          5|           21|         24|   Y|                N|              0.875|
|R1PXVYIYMYGUZL|          5|           21|         26|   Y|                N| 0.8076923076923077|
| RSFMJJXFZHX1C|          5|           24|         25|   Y|                N|               0.96|
|R1Z849RVP83BSJ|    

In [11]:
vine_5_star.count()

125

In [12]:
(vine_5_star.count()/ vine_votes_df.count())*100

46.99248120300752

In [13]:
non_vine_votes_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
non_vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|                1.0|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y| 0.9555555555555556|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y| 0.8873239436619719|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y| 0.9655172413793104|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y| 0.9142857142857143|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y| 0.9217391304347826|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y| 0.9705882352941176|
| RQCC7XNYI014B|    

In [14]:
non_vine_votes_df.count()

38829

In [15]:
non_vine_5_star = non_vine_votes_df.filter(non_vine_votes_df.star_rating == 5)
non_vine_5_star.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y| 0.9555555555555556|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y| 0.9655172413793104|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y| 0.9705882352941176|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|                1.0|
|R2KBV3A9X53ULG|          5|           39|         45|   N|                Y| 0.8666666666666667|
|R18XX6GVZ41CY1|          5|           21|         22|   N|                N| 0.9545454545454546|
|R3V8NHJWQWOU7S|          5|           27|         27|   N|                Y|                1.0|
|R16U59S8ZJGPLY|    

In [16]:
non_vine_5_star.count()

18246

In [17]:
(non_vine_5_star.count()/ non_vine_votes_df.count())*100

46.99065131731438